In [ ]:
import pandas as pd
import json
import os
from dotenv import load_dotenv

load_dotenv()

project_path = '/home/me/dev/spotanalysis'


raw_data = os.path.join(project_path, 'data', 'raw')
processed_data = os.path.join(project_path,'data', 'processed')
all_streams = []


 

print("---Getting Data---")
for filename in os.listdir(raw_data):
    if filename.startswith('Streaming_History_Audio') and filename.endswith('.json'):
        file_path = os.path.join(raw_data, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                all_streams.extend(data)
                print(f"Loaded: {filename}")
            except json.JSONDecodeError:
                print(f"Error reading JSON from {filename}. Skipping.")

# Convert dictionaries to pandas Data Frame
df_history = pd.DataFrame(all_streams)
print(f"\nTotal raw streams loaded: {len(df_history)}")

drop_columns = ['ip_addr', 'conn_country', 'platform', 'incognito_mode', ' offline_timestamp']
df_history.drop(columns=drop_columns, inplace=True, errors='ignore')
print("Sensitive columns scrubbed from data in memory.")

#convert endtime column to datetime endtime
df_history['ts'] = pd.to_datetime(df_history['ts'])

#Songs under 30s don't count as a stream. 
df_history_clean = df_history[df_history['ms_played'] >= 30000].copy()
print(f"Total streams after removing skips (<30s): {len(df_history_clean)}")

#cleaning 
df_history_clean['HourOfDay'] = df_history_clean['ts'].dt.hour
df_history_clean['DayOfWeek'] = df_history_clean['ts'].dt.day_name()
df_history_clean['Date'] = df_history_clean['ts'].dt.date

#Removing repeat songs
unique_tracks = df_history_clean[['master_metadata_album_artist_name', 'master_metadata_track_name']].drop_duplicates().reset_index(drop=True)

df_history_clean.to_csv(os.path.join(processed_data,'clean_history.csv'), index=False)
unique_tracks.to_csv(os.path.join(processed_data, 'unique_tracks.csv'), index=False)

print("saved clean history!")
print(f"Saved {len(unique_tracks)} unique tracks")

---Getting Data---
Loaded: Streaming_History_Audio_2021_8.json
Loaded: Streaming_History_Audio_2022_12.json
Loaded: Streaming_History_Audio_2021-2022_10.json
Loaded: Streaming_History_Audio_2023-2024_17.json
Loaded: Streaming_History_Audio_2019_1.json
Loaded: Streaming_History_Audio_2020_4.json
Loaded: Streaming_History_Audio_2020_3.json
Loaded: Streaming_History_Audio_2021_9.json
Loaded: Streaming_History_Audio_2021_6.json
Loaded: Streaming_History_Audio_2025_22.json
Loaded: Streaming_History_Audio_2024_18.json
Loaded: Streaming_History_Audio_2021_7.json
Loaded: Streaming_History_Audio_2020-2021_5.json
Loaded: Streaming_History_Audio_2024-2025_20.json
Loaded: Streaming_History_Audio_2024_19.json
Loaded: Streaming_History_Audio_2022_11.json
Loaded: Streaming_History_Audio_2023_14.json
Loaded: Streaming_History_Audio_2023_16.json
Loaded: Streaming_History_Audio_2019-2020_2.json
Loaded: Streaming_History_Audio_2023_15.json
Loaded: Streaming_History_Audio_2015-2019_0.json
Loaded: Streamin

In [ ]:
import pandas as pd
import os
import sys

# Spotify removed their API so external audio feature file was used from Kaggle
# External data does not have artist column so just merged with track names and dropped the dupes. S


CURRENT_DIR = os.getcwd() 


PROJECT_ROOT = os.path.dirname(CURRENT_DIR) 

# Define paths relative to the project root
PROCESSED_FOLDER = os.path.join(PROJECT_ROOT, 'data', 'processed')
RAW_FOLDER = os.path.join(PROJECT_ROOT, 'data', 'raw')

# Input File Paths
LOCAL_TRACKS_PATH = os.path.join(PROCESSED_FOLDER, 'unique_tracks.csv')
EXTERNAL_FEATURES_PATH = os.path.join(RAW_FOLDER, 'spot_features.csv') 

# Output File Path
OUTPUT_PATH = os.path.join(PROCESSED_FOLDER, 'enriched_tracks.csv')



# List of all the feature columns
FEATURE_COLS = [
    'danceability', 'energy', 'valence', 'loudness', 'mode', 'speechiness', 
    'acousticness', 'instrumentalness', 'liveness', 'tempo', 'key'
]

# Column Names for Local Files
LOCAL_ARTIST_COL = 'master_metadata_album_artist_name' 
LOCAL_TRACK_COL = 'master_metadata_track_name' 

# EXTERNAL FEATURE FILE (40k songs)
EXTERNAL_ARTIST_COL = 'genre' 
EXTERNAL_TRACK_COL = 'song_name' 


# Load Dataframes

try:
    
    local_cols_to_use = [LOCAL_ARTIST_COL, LOCAL_TRACK_COL]
    
    df_local = pd.read_csv(LOCAL_TRACKS_PATH, usecols=local_cols_to_use)
    print(f"Loaded local tracks: {len(df_local)} rows. (Optimized)")
    
    
    external_cols_to_use = [EXTERNAL_ARTIST_COL, EXTERNAL_TRACK_COL] + FEATURE_COLS
    
    
    df_external = pd.read_csv(EXTERNAL_FEATURES_PATH, usecols=external_cols_to_use)
    print(f"Loaded external features: {len(df_external)} rows. (Optimized)")

# error diagnosis
except FileNotFoundError as e:
    print(f"ERROR: Could not find a file. Please ensure the path is correct: {e}")
    print(f"Checked in project root: {PROJECT_ROOT}")
    

    try:
        print("\n--- DIAGNOSIS: Listing contents of the 'data/raw' folder ---")
        contents = os.listdir(RAW_FOLDER)
        print("Found files:")
        for item in contents:
            print(f"- {item}")
        print("\nACTION: Please check the list above for the correct feature file name (it might have different capitalization or spelling).")
        print("Then, update the variable 'EXTERNAL_FEATURES_PATH' in Step 1 and re-run.")
    except Exception as list_e:
        print(f"Could not list directory contents: {list_e}")
    
    
    sys.exit()
except ValueError as e:
    # Catches the ValueError related to missing columns in usecols 
    print(f"ERROR: Column loading failed (ValueError): {e}")
    print("\nFATAL ERROR: Failed to load external features. This should have been fixed by the previous step.")
    sys.exit()

except KeyError as e:
    # Catches KeyErrors if they occur later during processing
    print(f"ERROR: One of your configured column names is wrong: {e}")
    print("Please go back and update the variables in '--- 3. Configuration for Merge Keys ---'")
    sys.exit()


# Prep Merge Keys

# Lowercase local merge key
df_local['merge_key'] = (df_local[LOCAL_TRACK_COL].astype(str)).str.lower().str.strip()

# Lowercase external merge key
df_external['merge_key'] = (df_external[EXTERNAL_TRACK_COL].astype(str)).str.lower().str.strip()

# No dupes
df_external_features = df_external[['merge_key'] + FEATURE_COLS].drop_duplicates(subset=['merge_key'], keep='first')




# Use a LEFT merge to keep ALL of the local tracks
df_enriched = pd.merge(
    df_local,
    df_external_features,
    on='merge_key',
    how='left'
)


# Delete the large DataFrames that are no longer needed
del df_external
del df_local
del df_external_features # Delete temporary feature DF
print("\n[CLEANUP] Freed up memory from large input DataFrames.")



# Clean up the temporary merge key column
df_enriched = df_enriched.drop(columns=['merge_key'])

# Check and report success rate
matched_tracks = df_enriched['energy'].notna().sum()
total_tracks = len(df_enriched)
match_rate = (matched_tracks / total_tracks) * 100 if total_tracks > 0 else 0

print(f"\n" + "="*50)
print("DATA MERGE COMPLETE")
print(f"Total Unique Tracks: {total_tracks}")
print(f"Tracks enriched with features: {matched_tracks} ({match_rate:.2f}%)")
print(f"Final data shape: {df_enriched.shape}")
print("="*50)

# Save the final enriched DataFrame to the processed folder
df_enriched.to_csv(OUTPUT_PATH, index=False)
print(f"Final data saved to: {OUTPUT_PATH}")


Loaded local tracks: 8520 rows. (Optimized)


/tmp/ipykernel_11289/769254871.py:60: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_external = pd.read_csv(EXTERNAL_FEATURES_PATH, usecols=external_cols_to_use)


Loaded external features: 42305 rows. (Optimized)

[CLEANUP] Freed up memory from large input DataFrames.

DATA MERGE COMPLETE
Total Unique Tracks: 8520
Tracks enriched with features: 2379 (27.92%)
Final data shape: (8520, 13)
Final data saved to: /home/me/dev/spotanalysis/data/processed/enriched_tracks.csv
